In [1]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from tqdm import tqdm

# Suppress warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
sns.set_palette('rainbow')
sns.set_context('talk')

basedir = '/home/jack/Mounts/DiskOne/kona_coffee/imagemasks'

os.makedirs(basedir, exist_ok=True)

df = pd.read_csv('classified.csv')

df

,image,x,y,mask,lifecycle
0,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,299,58,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,ripening
1,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,324,300,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unknown
2,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,272,321,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unknown
3,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,364,288,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unknown
4,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,109,178,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,ripe
...,...,...,...,...,...
1200,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,160,294,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unripe
1201,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,176,343,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unripe
1202,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,116,241,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,unripe
1203,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,189,222,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,ripe


In [2]:
def get_display_image(row):
    cv2image = cv2.imread(row.image)

    mask = np.load(row.mask)
    mask = mask.astype(np.uint8)
    
    # apply the mask to the image
    mask_image = cv2.bitwise_and(cv2image, cv2image, mask=mask)
    
    # crop the image to the mask
    mask_image = mask_image[np.ix_(mask.any(1),mask.any(0))]
    
    return mask_image


fdf = []

for row in tqdm(df.itertuples(), total=len(df)):
    image_path = os.path.join(basedir, f'{row.image.split("/")[-1]}')

    if not os.path.exists(image_path):
        mask_image = get_display_image(row)
        cv2.imwrite(image_path, mask_image)
        
    fdf.append((image_path, row.lifecycle))
    
df = pd.DataFrame(fdf, columns=['image', 'lifecycle'])

# drop unknown and overripe lifecycles
df = df[df.lifecycle != 'unknown']
df = df[df.lifecycle != 'overripe']

df

100%|██████████| 1205/1205 [00:00<00:00, 20894.79it/s]


,image,lifecycle
0,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,ripening
4,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,ripe
5,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,ripening
6,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
7,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
...,...,...
1200,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
1201,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
1202,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,unripe
1203,/home/jack/Mounts/DiskOne/kona_coffee/imagemas...,ripe


In [3]:
df.to_csv('classified_masks.csv', index=False)